# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [1]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date, timedelta
from dotenv import load_dotenv
import os
import json

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [2]:
CITIES = ["Paris", "Berlin", "Madrid", "Amsterdam", "Vienna"]

In [3]:
### 💡 Première requête guidée

load_dotenv()
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5"

def get_weather_data(city):
    
    # URL : current weather data
    url = f"{OPENWEATHER_BASE_URL}/weather"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric',  # Celsius
        'lang': 'fr'
    }
    r = requests.get(url, params)
    print(r.status_code)
    print(r.json())
    # Votre code ici pour l'appel API
    # Gérez les erreurs HTTP !

In [4]:
for i in CITIES: 
    get_weather_data(i)

200
{'coord': {'lon': 2.3488, 'lat': 48.8534}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'couvert', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 17.81, 'feels_like': 17.89, 'temp_min': 16.88, 'temp_max': 19.09, 'pressure': 1020, 'humidity': 86, 'sea_level': 1020, 'grnd_level': 1011}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 300}, 'clouds': {'all': 100}, 'dt': 1753690525, 'sys': {'type': 1, 'id': 6550, 'country': 'FR', 'sunrise': 1753676318, 'sunset': 1753731320}, 'timezone': 7200, 'id': 2988507, 'name': 'Paris', 'cod': 200}
200
{'coord': {'lon': 13.4105, 'lat': 52.5244}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'nuageux', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 18.2, 'feels_like': 18.5, 'temp_min': 17.68, 'temp_max': 19.01, 'pressure': 1009, 'humidity': 93, 'sea_level': 1009, 'grnd_level': 1004}, 'visibility': 10000, 'wind': {'speed': 4.63, 'deg': 340}, 'clouds': {'all': 75}, 'dt': 1753691129, 'sys': {'type': 2, 'id': 203

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

- Si l'API retourne une erreur 401, cela correspond à une erreur de type "unauthenticated". Cela veut dire qu'on n'a pas renseigné notre clé API dans l'appel, ou que notre clé est invalide. Il faut donc renseigner notre clé ou créer une nouvelle clé valide.
- Une ville introuvable devrait nous renvoyer un status code 404 "not found".

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [5]:
# Configuration Visual Crossing
VCH_API_KEY = os.getenv("VCH_API_KEY")
VCH_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date):

    url = f"{VCH_BASE_URL}/{city}/{start_date}/{end_date}"

    params = {
        'key': VCH_API_KEY,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
    }
    # On fait l'API call
    r = requests.get(url, params)
    print(r.status_code)
    data_raw = r.json()

    # On récupère les données qui nous intéresse dans la réponse serveur
    data = data_raw.get("days", "clé inexistante")

    # On en fait un dataFrame
    df = pd.DataFrame(data)
    df.insert(0,'city', city)
    return df

    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

In [6]:
for i in CITIES: 
    get_historical_weather(i, "2025-7-25", "2025-7-27")

200
200
200
200
200


### 🎯 Mission pratique
Récupérez les données des 30 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [7]:
list_df = [get_historical_weather(i, "2025-7-21", "2025-7-27") for i in CITIES]
df_cities = [pd.concat(list_df, copy=False)]
print(df_cities[0])

200
200
200
200
200
        city    datetime  temp  humidity  precip  windspeed  pressure
0      Paris  2025-07-21  65.8      77.3   0.847       12.6    1004.2
1      Paris  2025-07-22  66.6      71.0   0.002       10.4    1013.4
2      Paris  2025-07-23  66.6      74.3   0.079        8.4    1013.8
3      Paris  2025-07-24  65.2      83.7   0.577        9.6    1014.9
4      Paris  2025-07-25  69.4      72.5   0.001        7.7    1017.7
5      Paris  2025-07-26  70.6      67.6   0.000       10.0    1018.2
6      Paris  2025-07-27  65.7      78.9   0.218       14.4    1016.4
0     Berlin  2025-07-21  68.6      84.0   0.934       18.3    1005.0
1     Berlin  2025-07-22  63.5      86.8   0.186       14.4    1006.1
2     Berlin  2025-07-23  65.1      83.8   1.153       15.5    1008.0
3     Berlin  2025-07-24  67.0      80.6   0.024       10.3    1011.3
4     Berlin  2025-07-25  67.8      85.3   0.166       12.8    1013.5
5     Berlin  2025-07-26  66.8      78.4   0.099       11.4    1014.8


In [16]:
### 🗺️ Obtenir les coordonnées
GEOCODING_API = "http://api.openweathermap.org/geo/1.0/direct"

def get_city_coordinates(city):
    
    # URL : current weather data
    url = f"{GEOCODING_API}"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'lang': "fr"
    }
    r = requests.get(url, params)
    print(r.status_code)
    data_raw = r.json()
    data = [data_raw[0].get("lat"),data_raw[0].get("lon")]
    return data
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """

In [40]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5"


def get_air_quality(city):

    # URL : pollution datas
    url = f"{OPENWEATHER_BASE_URL}/air_pollution"

    # Paramètres à compléter
    params = {
        'lat': get_city_coordinates(city)[0],
        'lon': get_city_coordinates(city)[1],
        'appid': OPENWEATHER_API_KEY,
        'lang': 'fr'
    }
    r = requests.get(url, params)
    print(r.status_code)
    data_raw = r.json()
    data = data_raw.get("list")
    air_data = {
        "air_components" : data[0].get("components"), 
        "indice aqi" : data[0].get("main").get("aqi")
    }
    return air_data

    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """

    

In [41]:
get_air_quality(CITIES[0])

200
200
200


{'air_components': {'co': 151.94,
  'no': 0.81,
  'no2': 2.36,
  'o3': 55.93,
  'so2': 0.78,
  'pm2_5': 3.11,
  'pm10': 4.5,
  'nh3': 2.17},
 'indice aqi': 1}


## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [ ]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [ ]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [ ]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites